To test out the JS Lemma, we can compute the nearest neighbors of a point and then find the rank of that matrix.

# Imports and Defaults

In [6]:
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from numpy.linalg import matrix_rank
from sklearn.neighbors import NearestNeighbors

In [8]:
path = '../writeup/images/q6'
seed = 0
rng = np.random.default_rng(seed)

# Intrinsic Dimension Estimator and Plotting Code

In [12]:
def est_intrinsic_dimension(X):
    n_pts, ambient_dim = X.shape
    
    # init nearest neighbors object
    neigh = NearestNeighbors(n_neighbors=2*ambient_dim).fit(X)
    nn_idx = neigh.kneighbors(return_distance=False) # [n_pts, n_neighbors]
    
    # iterate over nearest neighbors
    dim_est = []
    for idx in nn_idx:
        nn = X[idx] # nearest neighbors [n_neighbors, ambient_dim]
        rank = matrix_rank(nn)
        dim_est.append(rank) # rank of nearest neighbors estimates intrinsic dimension
    return np.mean(dim_est)
    

In [13]:
def plot(intrinsic_dim_list, est_dim_list, fname):
    # plot and save results
    fig1 = px.line(x=intrinsic_dim_list, y=est_dim_list)
    fig2 = px.scatter(x=intrinsic_dim_list, y=est_dim_list)
    fig3 = go.Figure(data=fig1.data + fig2.data).update_layout(
            xaxis_title="Intrinsic Dimension", yaxis_title="Estimated Dimension",
            margin_l=5, margin_t=5, margin_b=5, margin_r=5,
            font_family="Serif", font_size=14)
    fig3.show()
    fig3.write_image(os.path.join(path, fname))

# Estimate Intrinsic Dimension of Gaussians

In [4]:
def est_gaussian_dim(intrinsic_dim=4, ambient_dim=50, n_pts=5000):
    assert(intrinsic_dim <= ambient_dim)
    
    # original data X 
    mu = rng.uniform(low=-2, high=2, size=(1))
    sigma = np.eye(1)
    X = rng.multivariate_normal(mean=mu, cov=sigma, # [n_pts, intrinsic_dim]  
                                size=(n_pts, intrinsic_dim)
                                ).squeeze()
    
     # transformation matrix Y
    T = rng.multivariate_normal(mean=np.array([1]), cov=np.array([[1]]), # [intrinsic_dim, ambient_dim]
                                size=(intrinsic_dim, ambient_dim)
                                ).squeeze()
    
    # transformed data Y
    Y = X @ T # [n_pts, ambient_dim]
    
    # estimate dimension
    est_dim = est_intrinsic_dimension(Y)
    return est_dim

In [19]:
def test_gaussian():
    # estimate intrinsic dimension
    intrinsic_dim_list = [2, 4, 8, 16, 32]
    est_dim_list = [est_gaussian_dim(intrinsic_dim) for intrinsic_dim in intrinsic_dim_list]
    
    plot(intrinsic_dim_list, est_dim_list, 'gaussian.png')

test_gaussian()

# Estimate Intrinsic Dimension of Cubes

In [27]:
def est_hypercube_dim(intrinsic_dim=4, ambient_dim=50, n_pts=5000):
    assert(intrinsic_dim <= ambient_dim)
    
    # original data X 
    edge_length = rng.uniform(low=0, high=5, size=(1)).repeat(intrinsic_dim) # [intrinsic_dim]
    low, high = -edge_length/2, edge_length/2
    X = rng.uniform(low=low, high=high, size=(n_pts, intrinsic_dim)) # [n_pts, intrinsic_dim]
    
     # transformation matrix Y
    T = rng.multivariate_normal(mean=np.array([1]), cov=np.array([[1]]), # [intrinsic_dim, ambient_dim]
                                size=(intrinsic_dim, ambient_dim)
                                ).squeeze()
    
    # transformed data Y
    Y = X @ T # [n_pts, ambient_dim]
    
    # estimate dimension
    est_dim = est_intrinsic_dimension(Y)
    return est_dim

In [28]:
def test_hypercube():
    # estimate intrinsic dimension
    intrinsic_dim_list = [2, 4, 8, 16, 32]
    est_dim_list = [est_hypercube_dim(intrinsic_dim) for intrinsic_dim in intrinsic_dim_list]
    
    plot(intrinsic_dim_list, est_dim_list, 'hypercube.png')

test_hypercube()